Given a system (monomer, dimer), create a dictionary of <n> selection atom syntaxes that you can later on iterate over
    
If the system is a monomer, create selections for one protomer.
    
If the system is a dimer, have a dictionary for the monomer and for the protomer

In [1]:
#standard modules
import os
import glob

#import mdanalyis modules and packages
import MDAnalysis as mda
from MDAnalysis.analysis import rms, align, distances

#matplotlib
from matplotlib import pyplot as plt
import matplotlib.pylab as pylab
import matplotlib as mpl

#numpy
import numpy as np

#pandas
import pandas as pd

#nglview
import nglview

#seaborn
import seaborn as sns

#scipy
from scipy.stats import gaussian_kde

In [2]:
selection_ranges = {'protomer':[1,306],
                'domain1':[8,101],
                'lid':[40,55],
                'domain2':[102,184],
                'idl':[185,200],
                'domain3':[201,303],
                'nterm':[1,14],
                'cterm':[298,306],
                'oxyloop':[131,146],
               'oxyhole':[139,141]}

selection_sites = {'s1_pocket':[140,141,142,163,166,172],
         's1p_pocket':[24,25],
         's2_pocket':[41,49,54,165,187],
         's4_pocket':[165,167,192],
         'active_site':[41,145],
         'interface':[1,4,10,12,14,139,166,290,299],
                  'conserved':[2,5,9,13,14,16,18,20,
                               27,28,29,31,32,38,39,
                               40,41,66,75,77,87,95,
                               98,99,109,112,118,120,
                               131,135,136,139,140,143,
                               145,146,147,149,159,161,
                               163,166,167,172,174,179,
                               182,183,187,192,203,206,
                               208,209,210,211,231,234,
                               253,257,258,261,281,282,
                               289,290,296,299,302,303,306]}

In [15]:
def make_selection(selection,region):

    #syntax for a range of selections
    def make_range_selection(start,stop):  
        return f"resid {start}-{stop} and name CA"

    #syntax for selection sites
    def make_site_selection(sites):
        selection_string = ''
        for site in sites:
            selection_string += str(site) + ' '

        return 'resid '+ selection_string + 'and name CA'

    if region == 'range':
        #define selection in chain A
        range_a = selection_ranges[selection]
        sel_a = make_range_selection(range_a[0],range_a[1])

        #define selection in chain B
        range_b = []
        for sel in selection_ranges[selection]:
            range_b.append(sel + 306)

        sel_b = make_range_selection(range_b[0],range_b[1])

    elif region == 'site':
        #define selection in chain A
        sel_a = make_site_selection(selection_sites[selection])

        #define selection in chain B
        range_b = []
        for sel in selection_sites[selection]:
            range_b.append(sel + 306)
        sel_b = make_site_selection(range_b)

    return sel_a, sel_b

In [23]:
sites={}

for key,value in selection_ranges.items():
    sites[key] = make_selection(key,'range')
    
for key,value in selection_sites.items():
    sites[key] = make_selection(key,'site')


In [32]:
for key,value in sites.items():
    print(value[1])
    

resid 307-612 and name CA
resid 314-407 and name CA
resid 346-361 and name CA
resid 408-490 and name CA
resid 491-506 and name CA
resid 507-609 and name CA
resid 307-320 and name CA
resid 604-612 and name CA
resid 437-452 and name CA
resid 445-447 and name CA
resid 446 447 448 469 472 478 and name CA
resid 330 331 and name CA
resid 347 355 360 471 493 and name CA
resid 471 473 498 and name CA
resid 347 451 and name CA
resid 307 310 316 318 320 445 472 596 605 and name CA
resid 308 311 315 319 320 322 324 326 333 334 335 337 338 344 345 346 347 372 381 383 393 401 404 405 415 418 424 426 437 441 442 445 446 449 451 452 453 455 465 467 469 472 473 478 480 485 488 489 493 498 509 512 514 515 516 517 537 540 559 563 564 567 587 588 595 596 602 605 608 609 612 and name CA


In [31]:
site_map = pd.DataFrame.from_dict(sites)
site_map

,protomer,domain1,lid,domain2,idl,domain3,nterm,cterm,oxyloop,oxyhole,s1_pocket,s1p_pocket,s2_pocket,s4_pocket,active_site,interface,conserved
0,resid 1-306 and name CA,resid 8-101 and name CA,resid 40-55 and name CA,resid 102-184 and name CA,resid 185-200 and name CA,resid 201-303 and name CA,resid 1-14 and name CA,resid 298-306 and name CA,resid 131-146 and name CA,resid 139-141 and name CA,resid 140 141 142 163 166 172 and name CA,resid 24 25 and name CA,resid 41 49 54 165 187 and name CA,resid 165 167 192 and name CA,resid 41 145 and name CA,resid 1 4 10 12 14 139 166 290 299 and name CA,resid 2 5 9 13 14 16 18 20 27 28 29 31 32 38 3...
1,resid 307-612 and name CA,resid 314-407 and name CA,resid 346-361 and name CA,resid 408-490 and name CA,resid 491-506 and name CA,resid 507-609 and name CA,resid 307-320 and name CA,resid 604-612 and name CA,resid 437-452 and name CA,resid 445-447 and name CA,resid 446 447 448 469 472 478 and name CA,resid 330 331 and name CA,resid 347 355 360 471 493 and name CA,resid 471 473 498 and name CA,resid 347 451 and name CA,resid 307 310 316 318 320 445 472 596 605 and ...,resid 308 311 315 319 320 322 324 326 333 334 ...
